# SynthKernel

SynthKernel is a software architecture in TypeScript that enforces type safety, modularity and composability as well as making the APP easily extensible. It's rather a philosophy, a methodology and a set of standards than a bunch of static code.

SynthKernel leverages strengths in *Object-Oriented Programming*, *TypeScript Type System and Dynamic Nature*, *Facade Pattern*, and *Inversion of Control Principle*. It is born to fundamentally solve the problem of software unmaintainability by enforcing strict naming, file system, separation of concern and module composition standards without stifling logic flexibility. It also provides detailed patterns and templates on how to architect an application.

## Prerequisites

This whitepaper is a Jupyter Notebook running TypeScript. VSCode or any fork of it is recommended. to view it correctly, you need the following setup:

**Normal OS**:

Install Python3 and Node.js. After that, enter the following commands:

```sh
# install Jupyter if you haven't
pip3 install jupyterlab

# install TSLab to add TypeScript support, with your favourite package manager
npm install -g tslab

# register TS kernel
tslab install
```

Then open your VSCode and install extensions if not installed:

- `ms-toolsai.jupyter`
- `bierner.markdown-mermaid`

Clone this repo, go to this whitepaper, choose `Select Kernel` -> `Jupyter Kernel...` -> `TypeScript`, now you can read the notebook.

**Nix Approach**:

Install VSCode extensions if not installed:

- `ms-toolsai.jupyter`
- `bierner.markdown-mermaid`

Clone this repo and open your terminal **in the repo**, then enter command:

```sh
nix-shell shell.nix
```

Start VSCode **inside the terminal**, go to this whitepaper, choose `Select Kernel` -> `Jupyter Kernel...` -> `TypeScript`, now you can read the notebook.

## Architecture Overview

SynthKernel does impose very strict separation-of-concern disciplines, which ensures later maintainability and ease of codebase understanding once your understand SynthKernel. With separation, any project adopting SynthKernel can be easily visualized via a tree diagram and fit into file system constrains. Here's an example structure.

```mermaid
flowchart TD
    loader[loader]
    mod1[module]
    mod2[module]
    mod3[module]
    mod4[module]
    mod5[module]
    sub1[sub-loader]

    %% Connections
    loader --> mod1
    loader --> mod2
    loader --> sub1

    sub1 --> mod3
    sub1 --> mod4
    sub1 --> mod5
```

From the diagram, you can easily observe three types of nodes:

- loader: stem of the tree
- module: leaf of the tree
- sub-loader: branch of the tree

Loaders serve for three purposes - lifecycle manager of children modules, orchestrator of children-contributed types and the facade between the loader consumer and app's code. Loader itself does not do anything related to the app logic in most cases.

Modules are responsible for the app logic, they register lifecycle hooks, orchestrate types, wire each other via dependency injection, and augment their loader.

Sub-loaders can naturally emerge when your application has grown to a degree of complexity, where your find a module has been bloated too much. You can make the loader a new loader-module hierarchy to orchestrate the task.

## Mechanisms

### Inter-Module Communication

Complex application often require tight collaboration between different functionalities, this often leads to tight coupling. SynthKernel solves this via **module level dependency injection**.

SynthKernel modules are mostly singleton, to achieve inter-module communication, modules need to inject other modules. That is, modules are both service providers and subjects that is being injected to. All dependency injection happens autonomously at module level, while the loader simply registers all direct modules into a container that is passed to modules via constructor injection.

Combined with **module defined hooks**, dependency injection ensures loose coupling even in the most integrated functions, and makes any module can reach any part of the application like a traditional monolith. Since modules can only access modules explicitly registered in the container (often sibling modules and explicitly defined by the loader), this pattern can also ensure explicit dependency akin to React's unidirectional data flow.

### Lifecycle Hooks

Different applications require different lifecycle strategies. For example, a simple application may only have global construction and disposal, while a highly composable software may need to start and stop any module at any time. This generates lifecycle management requirements that cannot be managed in a single module. Loaders should manage in a collective manner.

Lifecycle hooks should be defined in the loader as simple subscription hooks like `onDispose` and `onModuleRestart`. The hooks should be passed to modules via constructor injection and then consumed by modules according to their needs.

### Base Module

Modules need a starting template so that they can implement their functions freely. A base module, often in companion with a loader, is inherited by all modules of the loader. The module should do some fixed tasks like receive DI container and lifecycle hooks passed from the loader.

More importantly, base modules serve as interfaces of type orchestration and augmentation declaration. And it also takes the role of type re-interpretation for some methods & properties of orchestrated types. This part will be elaborated later in [Type Orchestration](#type-orchestration) and [Augmentation](#augmentation).

### Type Orchestration

Highly modularized and extensible application often embodies poorly managed types, SynthKernel breaks the hell by declarative type orchestration via advanced generics.

For example, it is common for a module to define some configurable fields that can change the module's runtime behavior. Traditional application often uses a centralized configuration file to store these fields, which breaks the modularity by forcing modules to depend on a centralized configuration file. In SynthKernel, each module contribute their own atomic slices of configuration declaratively to the loader. Then the it merges these slices into a single object.

For example, module A can declare a configuration field like this:

```TypeScript
interface Config {
  a?: number,
};
```

And module B has config like this:

```TypeScript
interface Config {
  b: boolean,
};
```

The final config will be:

```TypeScript
interface Config {
  a?: number,
  b: boolean,
};
```

Moreover, it's common for the loader to pass some methods or properties that needs type mapping to the modules. For example, the loader receives the user defined configuration and then pass it to the modules. It cannot pass the config in exact type since the it is defined in generics. This creates a problem that the modules cannot receive the precise type even it they defined it themselves. To solve this, the type is intercepted by the base module, it augments the type using the module's type declaration. Then everything become accurately typed while is still modular and pluggable.

### Augmentation

Augmentation is another key component powered by SynthKernel's type orchestration capability. Without augmentation, the consumers need to access the DI container for APP functionalities while the loader can only do some basic lifecycle works. Augmentation allows modules to inject methods and properties back to the loader with full type safety, making it become a facade that encapsules app logic. The consumer can have accurate type hint as well as real runtime logic **on the loader** that comes from modules.

### Sub-Loader

Sub-loader is a combination of loader and module. Itself is designed to be a module of its parent loader, while it has grown to a degree of complexity that a new loader-module hierarchy is needed to orchestrate its functionality.

To create a sub-loader, simply create a new loader class as usual, then create another module class that instantiates the loader class. To put it another way, a sub-loader is a module plus a loader.

### Unidirectional Logic Flow

Sub-loader provides a modal of scalability by delegating growing complex logic to new self-sustaining units. However, this creates a hierarchy which comes with inter-hierarchal communication challenges. Bad design can still lead to unmaintainable code. Inspired by React, a logic flow standard is introduced to help build clear and explicit dependency relationships:

1. Each hierarchy is a self-sustaining unit with its isolated DI container.
2. Each module can only access its siblings and communicate with its parent.
3. To use a module in a higher hierarchy, it must be obtained from the parent DI container and provided into the child DI container by the sub-loader, this is exactly the same philosophy of setter and getter functions - they seem to be redundant, but they are helping you to keep the logic clean.
4. To subscribe module-defined hooks in a higher hierarchy, prefer to augment the sub-loader which is responsible for the subscription. Avoid directly subscribing to higher hooks in the module.

Above can be summarized as a unidirectional flow: **Raw down, fine up**.

What's down:

1. raw input
2. raw module logic

What's up:

1. selected methods and properties augmented to the loader
2. module-declared types to orchestrate

## End-to-End Example

In [ ]:
import { Container } from "@needle-di/core";

console.log("ss");

ss
